In [1]:
#import pyarrow.feather as feather
import pandas as pd
import numpy as np
import os
import seaborn as sns
sns.set(color_codes=True)
from numpy.random import seed
#from scipy.io import netcdf
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

from math import sqrt
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from keras.layers import Input, Dropout, Dense, LSTM, TimeDistributed, RepeatVector
from keras.models import Model
from keras import regularizers

In [2]:
# it is better for visualisation if the columns date type (not int or str)
# when save df to csv, if the column names are date, they are saved as str
def convert_col_to_date(columns):
    
    # convert the columns type
    #columns = df.columns
    new_columns = []
    for col in columns:
        try:
            new_columns.append(pd.to_datetime(col).date())
        except: new_columns.append(col)
    return new_columns

In [3]:
# import freq df
k=0 #sample number
freq_level =  pd.read_csv(f"dataset/modeling/sample{k}/freq_sample.csv").set_index("proj_id")
print(freq_level.shape)
print(" ")
print("details: ")
print(f"daily_freq: {freq_level[freq_level['daily']].shape}")
print(f"mid_weekly_freq: {freq_level[freq_level['mid_weekly']].shape}")
print(f"weekly_freq: {freq_level[freq_level['weekly']].shape}")
print(f"fortnight_freq: {freq_level[freq_level['fortnight']].shape}")
print(f"low_freq_freq: {freq_level[freq_level['low_freq']].shape}")

freq_level.sort_values(["freq", "length"])

(25, 7)
 
details: 
daily_freq: (5, 7)
mid_weekly_freq: (5, 7)
weekly_freq: (5, 7)
fortnight_freq: (5, 7)
low_freq_freq: (5, 7)


,daily,mid_weekly,weekly,fortnight,low_freq,length,freq
proj_id,,,,,,,
BY_22031,True,False,False,False,False,1239,1
BY_9294,True,False,False,False,False,1489,1
SH_10L53136015,True,False,False,False,False,3516,1
NI_40502502,True,False,False,False,False,5209,1
HB_30,True,False,False,False,False,5888,1
TH_4931210611,False,True,False,False,False,527,4
HE_15153,False,True,False,False,False,1107,4
NI_40501392,False,True,False,False,False,1810,4
SN_48400048,False,True,False,False,False,2408,4


In [4]:
# import t s and convert into date type
grouped_t_s = pd.read_csv(f"dataset/modeling/sample{k}/sample_t_s.csv").set_index("proj_id")
grouped_t_s.columns = convert_col_to_date(grouped_t_s.columns)
print(grouped_t_s.shape)
grouped_t_s.head()

(25, 6546)


,length,freq,1995-01-01,1995-01-02,1995-01-03,1995-01-04,1995-01-05,1995-01-06,1995-01-07,1995-01-08,...,2012-11-21,2012-11-22,2012-11-23,2012-11-24,2012-11-25,2012-11-26,2012-11-27,2012-11-28,2012-11-29,2012-11-30
proj_id,,,,,,,,,,,,,,,,,,,,,
BY_22031,1239,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,285.61398,285.595550,285.585870,285.573680,285.558280,285.549930,285.549930,285.54993,285.569290,285.633480
BY_9294,1489,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,490.71888,490.702060,490.710810,490.718370,490.726690,490.751960,490.771420,490.79915,490.820380,490.789480
SH_10L53136015,3516,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,30.55000,30.556667,30.543333,30.546667,30.566667,30.590000,30.606667,30.60000,30.606667,30.606667
NI_40502502,5209,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.62000,5.630000,5.620000,5.620000,5.620000,5.630000,5.630000,5.63000,5.630000,5.630000
HB_30,5888,1,NaN,3.17,3.16,3.15,3.1,3.07,3.04,3.01,...,2.71000,2.702500,2.700000,2.700000,2.738333,2.750833,2.768333,2.76500,2.759167,2.744167


In [5]:
# import ave_temp 
ave_temp = pd.read_csv(f"dataset/modeling/sample{k}/sample_temp.csv").set_index("proj_id")
ave_temp.columns = convert_col_to_date(ave_temp.columns)
print(ave_temp.shape)
ave_temp.head()


(25, 6546)


,length,freq,1995-01-01,1995-01-02,1995-01-03,1995-01-04,1995-01-05,1995-01-06,1995-01-07,1995-01-08,...,2012-11-21,2012-11-22,2012-11-23,2012-11-24,2012-11-25,2012-11-26,2012-11-27,2012-11-28,2012-11-29,2012-11-30
proj_id,,,,,,,,,,,,,,,,,,,,,
BY_22031,1239,1,9.0,-13.0,-33.0,-82.0,-88.0,-87.0,-80.0,-85.0,...,43.0,46.0,42.0,54.0,74.0,49.0,68.0,55.0,21.0,1.0
BY_9294,1489,1,9.0,-25.0,-36.0,-82.0,-101.0,-114.0,-69.0,-68.0,...,5.0,46.0,58.0,57.0,79.0,71.0,62.0,47.0,5.0,-10.0
SH_10L53136015,3516,1,6.0,-2.0,-24.0,-72.0,-73.0,-64.0,-85.0,-57.0,...,58.0,70.0,47.0,36.0,75.0,72.0,69.0,40.0,27.0,2.0
NI_40502502,5209,1,13.0,18.0,-5.0,-61.0,-67.0,-53.0,-62.0,-23.0,...,47.0,77.0,59.0,32.0,93.0,70.0,69.0,50.0,35.0,18.0
HB_30,5888,1,8.0,10.0,-19.0,-70.0,-75.0,-75.0,-62.0,-36.0,...,45.0,78.0,54.0,36.0,93.0,66.0,68.0,32.0,33.0,11.0


In [6]:
# import the cum_prec 
cum_prec=pd.read_csv(f"dataset/modeling/sample{k}/sample_prec.csv").set_index("proj_id")
cum_prec.columns = convert_col_to_date(cum_prec.columns)
print(cum_prec.shape)
cum_prec.head()

(25, 6546)


,length,freq,1995-01-01,1995-01-02,1995-01-03,1995-01-04,1995-01-05,1995-01-06,1995-01-07,1995-01-08,...,2012-11-21,2012-11-22,2012-11-23,2012-11-24,2012-11-25,2012-11-26,2012-11-27,2012-11-28,2012-11-29,2012-11-30
proj_id,,,,,,,,,,,,,,,,,,,,,
BY_22031,1239,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,62.0,22.0,138.0,91.0,0.0
BY_9294,1489,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,4.0,17.0,6.0,292.0,115.0,12.0
SH_10L53136015,3516,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,2.0,30.0,8.0,11.0,40.0,0.0,5.0,15.0,28.0
NI_40502502,5209,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,35.0,29.0,4.0,15.0,1.0,1.0,0.0,1.0
HB_30,5888,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,24.0,29.0,1.0,32.0,1.0,0.0,0.0,3.0


In [7]:
###########################################################################################################

# func

In [8]:
# create df for a t s (one row in grouped_t_s df)
def create_one_ts_df(row):
    bgr = row.name
    idx = row.dropna().index[2:]
    ts = row[idx]
    temp = ave_temp[idx].loc[bgr]
    prec = cum_prec[idx].loc[bgr]
    df = pd.DataFrame({"ts":ts,"temp": temp,"prec":prec, })
    return df



# define the autoencoder network model
def autoencoder_model(X):
    inputs = Input(shape=(X.shape[1], X.shape[2]))
    L1 = LSTM(16, activation='relu', return_sequences=True, 
              kernel_regularizer=regularizers.l2(0.00))(inputs)
    L2 = LSTM(4, activation='relu', return_sequences=False)(L1)
    L3 = RepeatVector(X.shape[1])(L2)
    L4 = LSTM(4, activation='relu', return_sequences=True)(L3)
    L5 = LSTM(16, activation='relu', return_sequences=True)(L4)
    output = TimeDistributed(Dense(X.shape[2]))(L5)    
    model = Model(inputs=inputs, outputs=output)
    return model

# find the outliers and save them in the df of a ts
def find_outlier(df):

    # split into train 80% and test sets, 
    train = df[:n_train]
    test = df[n_train:]

    # normalize the data
    X_train = scaler.fit_transform(train)
    X_test = scaler.transform(test)
    # reshape inputs for LSTM [samples, timesteps, features]
    X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
    #print("Training data shape:", X_train.shape)
    X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
    #print("Test data shape:", X_test.shape)

    # create the autoencoder model
    model = autoencoder_model(X_train)
    model.compile(optimizer='adam', loss='mae')
    #model.summary()

    # fit the model to the data
    history = model.fit(X_train, X_train, epochs=nb_epochs, batch_size=batch_size,validation_split=0.05).history
    
    # compute the threshold
    X_pred = model.predict(X_train)
    X_pred = X_pred.reshape(X_pred.shape[0], X_pred.shape[2])
    Xtrain = X_train.reshape(X_train.shape[0], X_train.shape[2])
    scored = np.mean(np.abs(X_pred-Xtrain), axis = 1)
    thre = np.mean(scored) +4*np.std(scored)
    print(thre)
    
    # find anomalies in original df
    pred_df  = model.predict(np.concatenate([X_train,X_test]))
    pred_df = pred_df.reshape(pred_df.shape[0], pred_df.shape[2])
    df["anomaly"] = np.mean(np.abs(pred_df -scaled_df), axis = 1)>thre


In [20]:
# global varieble
outlier = []
nb_epochs = 30
batch_size = 10
scaler = MinMaxScaler()


In [21]:
# apply on one ts
T1 = datetime.now()
print(f"T1: {T1}")
for i in range(10,14):
    ts= grouped_t_s.iloc[i]
    df = create_one_ts_df(ts)
    n_train = int(len(df)*0.8)
    scaled_df = scaler.fit_transform(df.values)
    # fit the train dataset
    find_outlier(df)
    outlier.append(df["anomaly"])

T2 = datetime.now()
print(f"T2: {T2}")
print(f"T2-T1: {T2-T1}")

T1: 2021-01-01 18:11:11.470562
Epoch 1/30
22/22 [==============================] - 5s 33ms/step - loss: 0.2811 - val_loss: 0.2884
Epoch 2/30
22/22 [==============================] - 0s 7ms/step - loss: 0.2603 - val_loss: 0.2691
Epoch 3/30
22/22 [==============================] - 0s 7ms/step - loss: 0.2471 - val_loss: 0.2500
Epoch 4/30
22/22 [==============================] - 0s 8ms/step - loss: 0.2240 - val_loss: 0.2331
Epoch 5/30
22/22 [==============================] - 0s 7ms/step - loss: 0.2265 - val_loss: 0.2238
Epoch 6/30
22/22 [==============================] - 0s 7ms/step - loss: 0.2018 - val_loss: 0.2172
Epoch 7/30
22/22 [==============================] - 0s 8ms/step - loss: 0.1880 - val_loss: 0.2089
Epoch 8/30
22/22 [==============================] - 0s 8ms/step - loss: 0.1791 - val_loss: 0.1935
Epoch 9/30
22/22 [==============================] - 0s 7ms/step - loss: 0.1553 - val_loss: 0.1563
Epoch 10/30
22/22 [==============================] - 0s 7ms/step - loss: 0.1226 - val_

52/52 [==============================] - 0s 5ms/step - loss: 0.1109 - val_loss: 0.0719
Epoch 24/30
52/52 [==============================] - 0s 5ms/step - loss: 0.1133 - val_loss: 0.0715
Epoch 25/30
52/52 [==============================] - 0s 5ms/step - loss: 0.1113 - val_loss: 0.0710
Epoch 26/30
52/52 [==============================] - 0s 5ms/step - loss: 0.1091 - val_loss: 0.0706
Epoch 27/30
52/52 [==============================] - 0s 5ms/step - loss: 0.1085 - val_loss: 0.0709
Epoch 28/30
52/52 [==============================] - 0s 5ms/step - loss: 0.1003 - val_loss: 0.0710
Epoch 29/30
52/52 [==============================] - 0s 5ms/step - loss: 0.1077 - val_loss: 0.0717
Epoch 30/30
52/52 [==============================] - 0s 5ms/step - loss: 0.0997 - val_loss: 0.0708
0.31798255870089753
Epoch 1/30
54/54 [==============================] - 6s 23ms/step - loss: 0.3417 - val_loss: 0.3348
Epoch 2/30
54/54 [==============================] - 0s 6ms/step - loss: 0.2842 - val_loss: 0.2623
Epo

In [18]:
# check for one ts
one_ts = pd.DataFrame((grouped_t_s.iloc[23].dropna(),outlier[3].dropna())).T
one_ts[one_ts["anomaly"]==True]

,NI_7216041,anomaly
1998-11-09,28.01,True


In [13]:
dd =pd.DataFrame(outlier)

In [14]:
dd

,1995-04-20,1995-10-20,1996-10-25,1997-04-20,1997-10-20,1998-04-15,1998-05-19,1998-06-17,1998-07-16,1998-08-17,...,2012-02-06,2012-03-07,2012-04-04,2012-05-07,2012-06-04,2012-07-03,2012-08-06,2012-09-03,2012-10-08,2012-11-06
anomaly,False,False,False,False,False,False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
anomaly,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
anomaly,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
anomaly,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
one_ts[one_ts["anomaly"].notnull()]

In [ ]:
outlier[2].shape

In [ ]:
outlier[0].shape